In [1]:
import os

import pandas as pd

In [2]:
movies = pd.read_csv(os.path.join('data', 'movies_raw.csv'), index_col='id')

In [3]:
movies.index.is_unique

False

In [4]:
movies.index.value_counts().sort_values(ascending=False)

94887      2
94989      2
96269      2
96267      2
96265      2
          ..
1136161    1
1136162    1
1136163    1
1136165    1
1136168    1
Name: id, Length: 893368, dtype: int64

There are duplicate movies IDs.

In [5]:
movies.loc[94887]

,title,genres,overview,popularity
id,,,,
94887,Girls Against Boys,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...","After a series of bad experiences with men, Sh...",7.305
94887,Girls Against Boys,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...","After a series of bad experiences with men, Sh...",7.305


In [6]:
movies.loc[94989]

,title,genres,overview,popularity
id,,,,
94989,The Five Cents of Lavarede,"[{'id': 12, 'name': 'Adventure'}, {'id': 35, '...",Some try to travel around the world with a tim...,1.341
94989,The Five Cents of Lavarede,"[{'id': 12, 'name': 'Adventure'}, {'id': 35, '...",Some try to travel around the world with a tim...,1.341


In [7]:
movies = movies.drop_duplicates()

In [8]:
movies.index.is_unique

True

In [9]:
movies.loc[94887]

title                                        Girls Against Boys
genres        [{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...
overview      After a series of bad experiences with men, Sh...
popularity                                                7.305
Name: 94887, dtype: object

In [10]:
movies.loc[94989]

title                                The Five Cents of Lavarede
genres        [{'id': 12, 'name': 'Adventure'}, {'id': 35, '...
overview      Some try to travel around the world with a tim...
popularity                                                1.341
Name: 94989, dtype: object

In [11]:
movies.to_csv(os.path.join('data', 'movies.csv'))